In [2]:
import os

import sqlite3
import pandas as pd

In [3]:
# Path to the root directory of datasci
dthon_path = os.path.join(os.path.expanduser('~'), 'Projects','datasci')
dthon_db = os.path.join(dthon_path, "melbourne-datathon","data","medi.db")
# Run this first to create database connection
conn = sqlite3.connect(dthon_db)
c = conn.cursor()

In [ ]:
# cOUNT total PATIENTS in vic WHO HAVE bought depression meds in 2015
sql = """
SELECT count(*)
FROM (
    SELECT Patient_ID
    FROM transactions
    WHERE Dispense_Week >= '2015-01-01'
    AND Dispense_Week < '2016-01-01'
    AND Drug_ID IN (
        SELECT c.MasterProductID AS Drug_ID
        FROM ChronicIllness_LookUp c
        WHERE c.ChronicIllness = 'Depression'
    )
    AND Patient_ID IN (
        SELECT Patient_ID
        FROM patients
        WHERE postcode IN (
            SELECT postcode FROM postcodes_geo
            WHERE state = 'VIC'
        )
    )
    GROUP BY Patient_ID
)

"""

%time cp = pd.read_sql_query(sql, conn)
cp.head()

In [ ]:
# cOUNT total PATIENTS WHO bought depression meds AND took tramadol in 2015
sql = """
SELECT COUNT(*)
FROM (
    SELECT Patient_ID
    FROM transactions t1
    WHERE Dispense_Week >= '2015-01-01'
    AND Dispense_Week < '2016-01-01'
    AND Patient_ID IN (
            SELECT Patient_ID
            FROM transactions
            WHERE t1.Dispense_Week = Dispense_Week
            AND Drug_ID IN (
                SELECT c.MasterProductID AS Drug_ID
                FROM ChronicIllness_LookUp c
                WHERE c.ChronicIllness = 'Depression'
            )
            AND Patient_ID IN (
                SELECT Patient_ID
                FROM patients
                WHERE postcode IN (
                    SELECT postcode FROM postcodes_geo
                    WHERE state = 'VIC'
                )
            )
            GROUP BY Patient_ID
    )
    AND Drug_ID IN (
            SELECT d.MasterProductID
            FROM Drug_LookUp d
            WHERE d.GenericIngredientName LIKE '%TRAMADOL%'
    )
    GROUP BY Patient_ID
)
"""

%time cp = pd.read_sql_query(sql, conn)
cp.head()

In [5]:
# cOUNT total PATIENTS WHO bought depression meds AND took tramadol in 2015
sql = """
SELECT COUNT(*)
FROM (
    SELECT t1.Patient_ID
    FROM trans_2015_vic_tram t1
    JOIN trans_2015_vic_dep t2 ON t1.Patient_ID = t2.Patient_ID AND t1.Dispense_Week = t2.Dispense_Week
    GROUP BY t1.Patient_ID
)
"""

%time cp = pd.read_sql_query(sql, conn)
cp.head()

CPU times: user 847 ms, sys: 45.8 ms, total: 892 ms
Wall time: 906 ms


,COUNT(*)
0,458


In [ ]:
# cOUNT total PATIENTS WHO bought PHENELZINE AND took tramadol in 2015
sql = """
SELECT COUNT(*)
FROM (
    SELECT Patient_ID
    FROM transactions
    WHERE Dispense_Week >= '2015-01-01'
    AND Dispense_Week < '2016-01-01'
    AND Patient_ID IN (
            SELECT Patient_ID
            FROM transactions
            WHERE Dispense_Week >= '2015-01-01'
            AND Dispense_Week < '2016-01-01'
            AND Drug_ID IN (
                SELECT c.MasterProductID AS Drug_ID
                FROM ChronicIllness_LookUp c
                WHERE c.ChronicIllness = 'Depression'
            )
            AND Patient_ID IN (
                SELECT Patient_ID
                FROM patients
                WHERE postcode IN (
                    SELECT postcode FROM postcodes_geo
                    WHERE state = 'VIC'
                )
            )
            GROUP BY Patient_ID
    )
    AND Drug_ID IN (
            SELECT d.MasterProductID
            FROM Drug_LookUp d
            WHERE d.GenericIngredientName LIKE '%TRAMADOL%'
    )
    GROUP BY Patient_ID
)
"""

%time cp = pd.read_sql_query(sql, conn)
cp.head()

In [ ]:
# This query gets pairs of transactions within two weeks of each other that may cause adverse affects
# based on the pairs in Adverse_Interaction table.
# Takes 5000 seconds or 83 minutes with one drug pair in adverse affects
sql = """
SELECT t1.Patient_ID,
    t1.Store_ID as store_id1, t1.Prescriber_ID as prescriber_id1, t1.Drug_ID as drug_id1, t1.Dispense_Week as dispense_week1,
    t2.Store_ID as store_id2, t2.Prescriber_ID as prescriber_id2, t2.Drug_ID as drug_id2, t2.Dispense_Week as dispense_week2
FROM tramadol_min_vic t1
JOIN  t2 ON t1.Patient_ID = t2.Patient_ID
AND t1.Dispense_Week = t2.Dispense_Week
AND t1.Drug_ID IN (
    SELECT MasterProductID
    FROM Drug_LookUp
    INNER JOIN Adverse_Interaction ai ON ai.DrugA_ID = GenericIngredientName
)
AND t2.Drug_ID IN (
    SELECT MasterProductID
    FROM Drug_LookUp
    INNER JOIN Adverse_Interaction ai ON ai.DrugA_ID IN (
        SELECT GenericIngredientName
        FROM Drug_LookUp
        WHERE MasterProductID = t1.Drug_ID
    )
    WHERE GenericIngredientName = ai.DrugB_ID
);

"""

%time df = pd.read_sql_query(sql, conn)
df.head()